In [72]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

In [87]:
from sklearn.datasets import fetch_20newsgroups
import enchant
import gensim
dict_eng = enchant.Dict('en')

df= pd.read_pickle("df")
text =np.asarray(df.text)
labels=np.asarray(df.category)
text, labels = shuffle(text, labels)

texts_train = text[:268]
texts_test = text[268:]
print(len(texts_train), len(texts_test))

268 30


In [88]:
labels_train = labels[:268]
labels_test = labels[268:]

In [89]:
texts_train[0]

'(CNN) When Sonya Kennedy learned that her 12-year-old son, Ryder, had tried vaping, she was "mortified."\n\n"He honestly didn\'t think that there was anything wrong," Kennedy said. "He told me that almost all the grade-sevens were doing it."\n\nAnd she soon realized that the trend had reached many more kids in her orbit: Kennedy, who owns a dance studio in Northern California, found that most of her dancers had tried vaping, too. She learned that kids were vaping at school, taking puffs inside their shirts and in some cases charging their e-cigarettes in their teacher\'s own computer, she said.\n\nMeanwhile, many parents seemed oblivious.\n\n"It\'s a bad habit that I started realizing parents didn\'t know about," she said.\n\nHealth experts say parents whose kids are vaping often don\'t know what to do or where to turn for help.\n\nWhile federal authorities grapple with how to regulate e-cigarettes on a broad scale and while leading tobacco company Altria invests billions in e-cigaret

In [90]:
#чистка
texts_prep_train = []

for text in log_progress(texts_train):
    filtered = re.findall('(?u)\\b\\w\\w+\\b', text)
    filtered = [filt for filt in filtered if dict_eng.check(filt) and filt not in ['s', 't', 'd', 'v', 've', 'll', 're']]
    texts_prep_train.append(' '.join(filtered).lower())
texts_prep_train[0]

'cnn when sonya kennedy learned that her 12 year old son ryder had tried vaping she was mortified he honestly think that there was anything wrong kennedy said he told me that almost all the grade sevens were doing it and she soon realized that the trend had reached many more kids in her orbit kennedy who owns dance studio in northern california found that most of her dancers had tried vaping too she learned that kids were vaping at school taking puffs inside their shirts and in some cases charging their cigarettes in their teacher own computer she said meanwhile many parents seemed oblivious it bad habit that started realizing parents know about she said health experts say parents whose kids are vaping often don know what to do or where to turn for help while federal authorities grapple with how to regulate cigarettes on broad scale and while leading tobacco company invests billions in cigarette maker parents are scrambling at home to deal with nicotine dependence or to prevent their k

In [91]:
texts_prep_test = []

for text in log_progress(texts_test):
    filtered = re.findall('(?u)\\b\\w\\w+\\b', text)
    filtered = [filt for filt in filtered if dict_eng.check(filt) and filt not in ['s', 't', 'd', 'v', 've', 'll', 're']]
    texts_prep_test.append(' '.join(filtered).lower())

In [92]:
#токенизация
corpus_train = []
labels_train_ = []

for text, lbl in zip(log_progress(texts_prep_train), labels_train):
    tokens = list(gensim.utils.tokenize(text, lower=True))
    if len(tokens) != 0:
        corpus_train.append(tokens)
        labels_train_.append(lbl)
np.transpose(corpus_train[0])

array(['cnn', 'when', 'sonya', ..., 'this', 'as', 'journey'], dtype='<U15')

In [93]:
corpus_test = []
labels_test_ = []
for text, lbl in zip(log_progress(texts_prep_test), labels_test):
    tokens = list(gensim.utils.tokenize(text, lower=True))
    if len(tokens) != 0:
        corpus_test.append(tokens)
        labels_test_.append(lbl)

In [94]:
#тэггирование документа
documents_train = [gensim.models.doc2vec.TaggedDocument(tokens, [i]) for i, tokens in enumerate(corpus_train)]
documents_test = [gensim.models.doc2vec.TaggedDocument(tokens, [i]) for i, tokens in enumerate(corpus_test)]

documents_train[0]

TaggedDocument(words=['cnn', 'when', 'sonya', 'kennedy', 'learned', 'that', 'her', 'year', 'old', 'son', 'ryder', 'had', 'tried', 'vaping', 'she', 'was', 'mortified', 'he', 'honestly', 'think', 'that', 'there', 'was', 'anything', 'wrong', 'kennedy', 'said', 'he', 'told', 'me', 'that', 'almost', 'all', 'the', 'grade', 'sevens', 'were', 'doing', 'it', 'and', 'she', 'soon', 'realized', 'that', 'the', 'trend', 'had', 'reached', 'many', 'more', 'kids', 'in', 'her', 'orbit', 'kennedy', 'who', 'owns', 'dance', 'studio', 'in', 'northern', 'california', 'found', 'that', 'most', 'of', 'her', 'dancers', 'had', 'tried', 'vaping', 'too', 'she', 'learned', 'that', 'kids', 'were', 'vaping', 'at', 'school', 'taking', 'puffs', 'inside', 'their', 'shirts', 'and', 'in', 'some', 'cases', 'charging', 'their', 'cigarettes', 'in', 'their', 'teacher', 'own', 'computer', 'she', 'said', 'meanwhile', 'many', 'parents', 'seemed', 'oblivious', 'it', 'bad', 'habit', 'that', 'started', 'realizing', 'parents', 'know'

In [95]:
model = gensim.models.Doc2Vec(documents_train, vector_size=100, window=5, min_count=5, workers=4)

In [96]:
#векторизация
vectors_train = []

for tokens in corpus_train:
    vectors_train.append(model.infer_vector(tokens))
    
vectors_train = np.asarray(vectors_train)
print(vectors_train.shape)

(268, 100)


In [97]:
vectors_test = []

for tokens in corpus_test:
    vectors_test.append(model.infer_vector(tokens))
    
vectors_test = np.asarray(vectors_test)
print(vectors_test[0])

[-0.38997683  0.07719938 -0.07460335  0.88343245 -0.3908429   0.6052178
  0.26082906 -0.8945105   0.2344684   0.08588755  0.08913531 -0.61178845
  0.05141707 -0.00989374 -0.06665654 -0.90481395 -0.973004    0.06221562
  0.651137   -0.00646208 -0.26703575  0.08904139 -0.13408048  0.07400348
 -0.2266136   0.09570494  0.18352142 -0.500869    0.13472341  0.7595682
 -0.03802006  0.11134852  0.1838541  -0.5060521  -0.02048611 -0.32058313
 -0.4182475  -0.08979752  0.6265962   0.6729039   0.06352419 -0.2992014
 -0.11024582 -0.6779646  -0.25742337 -0.4359367  -0.5919347  -0.1493966
  0.684934    0.09499463 -0.04029176  0.2612821  -0.10788965  0.26944116
 -0.27248693  0.06739323 -0.15811615 -0.43019265 -0.06772966 -0.06054794
 -0.01411101 -0.31703174 -0.16513517  0.16114089  0.1016127   0.3964094
  0.373097   -0.06341266 -0.52869415  0.67630655 -0.148711    0.17416161
  0.4452954  -0.24812636 -0.07785248 -0.6474222   0.04521478  0.3047361
  0.5363902  -0.56960297  0.02884387  0.32569644  0.04839

In [98]:
from sklearn.ensemble import RandomForestClassifier
predictor = RandomForestClassifier(n_estimators=100).fit(vectors_train, labels_train_)
preds = predictor.predict(vectors_test)

In [99]:
np.mean(preds == labels_test_)

0.7666666666666667

In [100]:
print(preds)
print(np.asarray(labels_test_))

['asia' 'travel' 'health' 'sport' 'health' 'sport' 'travel' 'sport'
 'travel' 'health' 'health' 'travel' 'travel' 'health' 'health' 'health'
 'health' 'health' 'travel' 'travel' 'sport' 'travel' 'health' 'sport'
 'travel' 'health' 'travel' 'sport' 'health' 'travel']
['travel' 'travel' 'health' 'travel' 'health' 'travel' 'travel' 'sport'
 'travel' 'health' 'health' 'health' 'asia' 'health' 'health' 'health'
 'health' 'health' 'travel' 'health' 'asia' 'travel' 'health' 'sport'
 'travel' 'health' 'travel' 'sport' 'health' 'travel']


In [23]:
import re
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')